# Classificação de objetos

> Atenção: este notebook foi desenhado para funcionar no **Google Collab**. Se pretende executar localmente prefira a versão local deste notebook, sem o sufixo ```-collab```.

## 1. Requerimentos

Todas as bibliotecas já estão instaladas no Google Colab.

* OpenCV >= 3.4.3
* Matplotlib >= 3.1.3
* Seaborn >= 0.0.10
* Numpy >= 1.18.1

### 1.2 Arquivos

Baixe o repositório do GitHub utilizando o comando abaixo. Em caso de atualização, utilize o comando para apagar o diretório antes.

In [ ]:
!rm -rf fiap-ml-visao-computacional/

In [ ]:
!git clone https://github.com/michelpf/fiap-ml-visao-computacional

Vamos agora posicionar o diretório do repositório para a aula respectiva. Nesse caso envie o comando de mudança de diretório.

In [ ]:
%cd fiap-ml-visao-computacional/aula-4-classificacao-objetos-analise-facial/

Importação das bibliotecas.

In [ ]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

#Exibição na mesma tela do Jupyter
%matplotlib inline

import datetime
import dlib
from scipy.spatial import distance as dist

plt.style.use('seaborn')
sns.set_style("whitegrid", {'axes.grid' : False})

Carregando um classificador pré-treinado de Haar.

Analise posteriormente outros classificadores disponíveis, dentre eles, classificador de pessoas, automóveis, gatos, sorriso, olhos, etc. neste repositório oficial do OpenCV https://github.com/opencv/opencv/tree/master/data/haarcascades.

## 2. Classificador de Viola-Jones

Este classificador é especializado em identificar faces, também é conhecido como classificador Viola James e pode ser utilizado em outras aplicações. O paper original pode ser baixado [aqui](https://www.cs.cmu.edu/~efros/courses/LBMV07/Papers/viola-cvpr-01.pdf). 

Devido a característica deste tipo de classificador, sua identificação é extramamente rápida, com identificação < 0,02s, aplicações em sistemas em tempo real, especialmente câmeras de vigilância.

### 2.1 Classificador de Faces

A aplicação mais utilizada do classificador em cascata de Viola-Jones (ou classificador de Haar) é para encontrar faces em imagens ou vídeos, especialmente por sua identificação ser bem rápida.

Este tipo de classificador serve para separar a região de interesse de uma determinada área, para posteriormente, aplicar outros classificadores que poderão, por exemplo, classificar de quem é o rosto, uma vez que a região foi separado da imagem original.

In [ ]:
# Carregando classifcador
classificador_face = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')

imagem = cv2.imread('imagens/people.jpg')

imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
imagem_gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(imagem)
plt.axis("off")

In [ ]:
# Retornará a região de interesse da face identificada como tupla, armazenando as coordenadas superiores esquerda e inferior 
# direita.
# Se retornar vazio é por que não há faces identificadas.
# Os valores padrão são configurações inciais recomendadas 
# (cv.HaarDetectObjects(image, cascade, storage, scale_factor=1.1, min_neighbors=3, flags=0, min_size=(0, 0)))

faces = classificador_face.detectMultiScale(imagem_gray, 1.3, 5)

# Lista de faces. Caso não seja identificada será retornado None (nulo)
if faces is None:
     cv2.putText(imagem, "Rosto ausente", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 6)
    
# Desenhando retângulos nos rostos identificados
for (x,y,w,h) in faces:
    cv2.rectangle(imagem, (x,y), (x+w,y+h), (255,255,0), 2)
    
plt.figure(figsize=(20,10))
plt.imshow(imagem)
plt.axis('off')
plt.title("Faces identificadas")

Construindo funções para utilizar de forma modular em análise de imagens ou vídeos. 

Optamos por criar 2 versões da mesma função, sendo a com o sufixo **rgb** espera como parâmetro de entrada uma imagem neste formato. A outra função espera como imagem de entrada no formato **bgr**, que é o formato padrão para escrita de vídeo.

Função de análise e inferência para imagens ou frames no formato RGB.

In [ ]:
def identificar_rosto_rgb(imagem):
    classificador_face = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')

    imagem_gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    faces = classificador_face.detectMultiScale(imagem_gray, 1.2, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(imagem, (x,y), (x+w,y+h), (255,255,0), 2)

    return imagem

Função de análise e inferência para imagens ou frames no formato BGR.

In [ ]:
def identificar_rosto_bgr(imagem):
    classificador_face = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')

    imagem_gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    faces = classificador_face.detectMultiScale(imagem_gray, 1.2, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(imagem, (x,y), (x+w,y+h), (0,255,255), 2)

    return imagem

Aplicação do classificador em cascata para leitura e processamento de vídeo. A análise é realizada frame-a-frame e o resultado do processamento é salvo em outro vídeo para posteiror verificação.
Como a escrita de arquivos é feita no formato BGR, optamos por usar a função com este sufixo (bgr) para o processamento. Poderíamos usar a funçÃo RGB, mas nesse caso precisaríamos de converter cada frame para RGB e depois para BGR, o que consumiria muito processamento.

In [ ]:
video = cv2.VideoCapture("videos/people-talking.mp4")
video_fps = video.get(cv2.CAP_PROP_FPS)

print("Frames per second (FPS): " + str(video_fps))

writer = None
fourcc = cv2.VideoWriter_fourcc(*"MJPG")

while True:
    ret, frame = video.read()
    if ret:
        frame = identificar_rosto_bgr(frame)
        if writer is None:
            writer = cv2.VideoWriter("videos/people-talking-processed.avi", fourcc, video_fps, (frame.shape[1], frame.shape[0]), True)
        writer.write(frame)
    else:
        break
        
video.release()

Aplicando a uma imagem coletada de uma câmera.

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo("imagens/foto.jpg")
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
imagem = cv2.imread("imagens/foto.jpg")
imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
imagem_rosto = identificar_rosto_rgb(imagem)

plt.figure(figsize=(20,10))
plt.imshow(imagem_rosto, cmap="gray")
plt.title("Imagem com rosto")

### 2.2 Classificador em Cascata Customizado

Construir um classificador em cascata especializado consiste em reunir imagens de treino positivas e negativas, diferentemente de outros tipos de classificadores, como por exemplo os de _deep learning_. 

O processo de construção é realizado por meio de utilitários em linha de comando do OpenCV, que apesar de não ser complexo pode envolver uma dedicação maior. Por tal razão, vamos optar pelo [Cascade Trainer GUI](http://amin-ahmadi.com/cascade-trainer-gui/), de Amin Ahmadi, criado para ser executado em Windows que facilita muito este processo. http://amin-ahmadi.com/cascade-trainer-gui/.

As imagens para as etapas de treino podem ser baixadas em http://www.dis.uniroma1.it/~labrococo/?q=node/459. 

Função de análise e inferência para imagens ou frames no formato RGB.

In [ ]:
def identificar_bola_rgb(imagem):
    ball_classifier = cv2.CascadeClassifier('ball/classifier/cascade.xml')
  
    gray = cv2.cvtColor(imagem, cv2.COLOR_RGB2GRAY)
    balls = ball_classifier.detectMultiScale(gray, 1.3, 3)

    for (x,y,w,h) in balls:
        roi = imagem[y:y+h, x:x+w]
        area = int(w) * int(h)
        
        if area > 1500:
            cv2.rectangle(imagem, (x,y), (x+w,y+h), (255,255,0), 1)
        
    return imagem

Função de análise e inferência para imagens ou frames no formato BGR.

In [ ]:
def identificar_bola_bgr(imagem):
    ball_classifier = cv2.CascadeClassifier('ball/classifier/cascade.xml')
  
    gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    balls = ball_classifier.detectMultiScale(gray, 1.3, 3)
    
    for (x,y,w,h) in balls:
        roi = imagem[y:y+h, x:x+w]
        area = int(w) * int(h)
        
        if area > 1500:
            cv2.rectangle(imagem, (x,y), (x+w,y+h), (0,255,255), 1)
        
    return imagem

In [ ]:
imagem = cv2.imread("imagens/soccer-ball.png")
imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(20,10))
plt.imshow(imagem)
plt.axis('off')
plt.title("Bola de futebol")

In [ ]:
imagem_proc = identificar_bola_rgb(imagem)

plt.figure(figsize=(20,20))
plt.imshow(imagem_proc)
plt.axis('off')
plt.title("Bola de futebol")

Processamento de vídeo utilizando o classificador customizado. Neste vídeo temos os mesmos elementos utilizados na classificação, como a bola (objeto classificado, positivo) e o fundo (objeto não desejado, faz parte do conjunto de imagens negativas).

In [ ]:
video = cv2.VideoCapture("videos/soccer.avi")
video_fps = video.get(cv2.CAP_PROP_FPS)

print("Frames per second (FPS): " + str(video_fps))

writer = None
fourcc = cv2.VideoWriter_fourcc(*"MJPG")

while True:
    ret, frame = video.read()
    if ret:
        frame = identificar_bola_bgr(frame)

        if writer is None:
            writer = cv2.VideoWriter("videos/soccer-processed.avi", fourcc, video_fps, (frame.shape[1], frame.shape[0]), True)
        writer.write(frame)
    else:
        break
        
video.release()

### 2.3 Classificador de Pessoas

Este classificador identifica pessoas (corpos inteiros). Sua utilidade é para segmentar e localizar uma ou mais pessoas em uma determinada cena.

Repare que pessoas muito próximas uma das outras prejudica a identificação. Em vídeos de vigilância, este aspecto negativo é diminuído, pois ha muitas oportunidades do classificador inferir, e conseguir identificar as pessoas em outros momentos onde estão mais separadas umas das outroas.

Função de análise e inferência para imagens ou frames no formato RGB.

In [ ]:
def identificar_pessoas_rgb(imagem):
    classificador_pessoas = cv2.CascadeClassifier('classificadores/haarcascade_fullbody.xml')
    
    imagem_gray = cv2.cvtColor(imagem, cv2.COLOR_RGB2GRAY)
    pessoas = classificador_pessoas.detectMultiScale(imagem_gray, 1.2, 3)

    for (x,y,w,h) in pessoas:
        roi = imagem[y:y+h, x:x+w]
        area = int(w) * int(h)
        cv2.rectangle(imagem, (x,y), (x+w,y+h), (255,255,0), 2)
        
    return imagem

Função de análise e inferência para imagens ou frames no formato BGR.

In [ ]:
def identificar_pessoas_bgr(imagem):
    classificador_pessoas = cv2.CascadeClassifier('classificadores/haarcascade_fullbody.xml')
    
    imagem_gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    pessoas = classificador_pessoas.detectMultiScale(imagem_gray, 1.2, 3)

    for (x,y,w,h) in pessoas:
        roi = imagem[y:y+h, x:x+w]
        area = int(w) * int(h)
        cv2.rectangle(imagem, (x,y), (x+w,y+h), (0,255,255), 2)
        
    return imagem

In [ ]:
imagem = cv2.imread("imagens/people-walking.png")
imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(20,10))
plt.imshow(imagem)
plt.axis('off')
plt.title("Pessoas")

In [ ]:
imagem_proc = identificar_pessoas_rgb(imagem)

plt.figure(figsize=(20,10))
plt.imshow(imagem_proc)
plt.axis('off')
plt.title("Pessoas")

Aplicando vídeo para classificação.

In [ ]:
video = cv2.VideoCapture("videos/walking.avi")
video_fps = video.get(cv2.CAP_PROP_FPS)

print("Frames per second (FPS): " + str(video_fps))

writer = None
fourcc = cv2.VideoWriter_fourcc(*"MJPG")

while True:
    ret, frame = video.read()
    if ret:
        frame = identificar_pessoas_bgr(frame)
        if writer is None:
            writer = cv2.VideoWriter("videos/people-walking-processed.avi", fourcc, video_fps, (frame.shape[1], frame.shape[0]), True)
        writer.write(frame)
    else:
        break
        
video.release()

### 2.4. Classificador em Cascata de Automóveis

Classificador especializado em identificar automóveis.

Função de análise e inferência para imagens ou frames no formato RGB.

In [ ]:
def identificar_carros_rgb(imagem):
    cars_classifier = cv2.CascadeClassifier('classificadores/haarcascade_car.xml')
  
    gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    cars = cars_classifier.detectMultiScale(gray, 1.2, 3)

    for (x,y,w,h) in cars:
        roi = imagem[y:y+h, x:x+w]
        area = int(w) * int(h)
        cv2.rectangle(imagem, (x,y), (x+w,y+h), (255,255,0), 1)
        
    return imagem

Função de análise e inferência para imagens ou frames no formato BGR.

In [ ]:
def identificar_carros_bgr(imagem):
    cars_classifier = cv2.CascadeClassifier('classificadores/haarcascade_car.xml')
  
    gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    cars = cars_classifier.detectMultiScale(gray, 1.2, 3)

    for (x,y,w,h) in cars:
        roi = imagem[y:y+h, x:x+w]
        area = int(w) * int(h)
        cv2.rectangle(imagem, (x,y), (x+w,y+h), (0,255,255), 1)
        
    return imagem

In [ ]:
imagem = cv2.imread("imagens/cars.png")
imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(20,10))
plt.axis('off')
plt.imshow(imagem)
plt.title("Carros")

In [ ]:
imagem_proc = identificar_carros_rgb(imagem)

plt.figure(figsize=(30,20))
plt.imshow(imagem_proc)
plt.axis('off')
plt.title("Carros")

Análise de vídeo de câmera de uma estrada e inferência do classificador.

In [ ]:
video = cv2.VideoCapture("videos/cars.avi")
video_fps = video.get(cv2.CAP_PROP_FPS)

print("Frames per second (FPS): " + str(video_fps))

writer = None
fourcc = cv2.VideoWriter_fourcc(*"MJPG")

while True:
    ret, frame = video.read()
    if ret:
        frame = identificar_carros_bgr(frame)
        if writer is None:
            writer = cv2.VideoWriter("videos/cars-processed.avi", fourcc, video_fps, (frame.shape[1], frame.shape[0]), True)
        writer.write(frame)
    else:
        break
        
video.release()

## 3. Identificação de Marcos Faciais

O toolkit [Dlib](http://dlib.net) fornece uma série de bibliotecas e ferramentas para utilização em problemas de visão computacional. Para o uso de identificação de marcos faciais vamos utilizar sua biblioteca responsável por identificar faces de uma imagem e posterior identificação dos marcos faciais.

O Dlib vem instalado por padrão na plataforma Google Colab, não é necesário baixar.

Os modelos podem ser baixados neste endereço http://dlib.net/files/. Os que serão utilizados estão já incluídos na pasta ```modelos```.

Iniciamos o processo de identificação configurando o modelo treinado de 68 pontos ```shape_predictor_68_face_landmarks.dat```. Após isso precisamos identificar a face e para cada face identificada retornar a lista de pontos, na variável ```marcos_faciais```.

In [ ]:
classificador_68_path = "modelos/shape_predictor_68_face_landmarks.dat"
classificador_5_path = "modelos/shape_predictor_5_face_landmarks.dat"

classificador_dlib_68 = dlib.shape_predictor(classificador_68_path)
classificador_dlib_5 = dlib.shape_predictor(classificador_5_path)

detector_face_dlib = dlib.get_frontal_face_detector()

def obter_marcos(imagem, marcos_68_pontos=True):
    
    classificador_dlib = classificador_dlib_68
    
    if marcos_68_pontos is False:
        classificador_dlib = classificador_dlib_5
        
    faces = detector_face_dlib(imagem, 1)

    if len(faces) == 0:
        print("Não foi encontrada nenhuma face.")
        return None
        
    marcos_faciais = []
    
    for face in faces:
        marcos_faciais.append(np.matrix([[p.x, p.y] for p in classificador_dlib(imagem, face).parts()]))

    return marcos_faciais

Com os pontos mapeados, o próximo passo é criar uma função para anotar em uma nova imagem.

In [ ]:
def anotar_marcos(imagem, marcos_faciais):
    
    if marcos_faciais is None:
        print("Não foi identificado nenhum marco facial.")
        return imagem
    
    for marco_facial in marcos_faciais:
        for idx, ponto in enumerate(marco_facial):
            centro = (ponto[0,0], ponto[0,1])
            
            cv2.putText(imagem, str(idx), centro, fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                        fontScale=0.5, color=(255, 255, 255))
            cv2.circle(imagem, centro, 3, color=(0, 255, 255), thickness=-1)
            
    return imagem

In [ ]:
imagem = cv2.imread("imagens/girl.jpg")

marcos_faciais = obter_marcos(imagem)
imagem_marcos = anotar_marcos(imagem, marcos_faciais)

imagem_marcos = cv2.cvtColor(imagem_marcos, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(30,20))
plt.axis('off')
plt.imshow(imagem_marcos)
plt.title("Marcos faciais com 68 pontos")

In [ ]:
imagem = cv2.imread("imagens/girl.jpg")

marcos_faciais = obter_marcos(imagem, False)
imagem_marcos = anotar_marcos(imagem, marcos_faciais)

imagem_marcos = cv2.cvtColor(imagem_marcos, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(30,20))
plt.axis('off')
plt.imshow(imagem_marcos)
plt.title("Marcos faciais com 5 pontos")

Aplicando a detecção e anotação utilizando uma câmera.

In [ ]:
from IPython.display import Image
try:
  filename = take_photo("imagens/foto.jpg")
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
imagem = cv2.imread("imagens/foto.jpg")

marcos_faciais = obter_marcos(imagem)
imagem_marcos = anotar_marcos(imagem, marcos_faciais)

imagem_marcos = cv2.cvtColor(imagem_marcos, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(30,20))
plt.axis('off')
plt.imshow(imagem_marcos)
plt.title("Marcos faciais da câmera")

### 3.1 Análise de Marcos Faciais

Os 68 pontos estão divididos nos seguintes coponentes do rosto humano:

* Face completa ```1 ao 68```
* Sombrancelha direita ```17 ao 21```
* Sombracelha esquerda ```22 ao 26```
* Olho direito ```36 ao 41```
* Olho esquerdo ```42 ao 47```
* Nariz ```27 ao 34```
* Lábio ```48 ao 60```
* Mandíbula ```1 ao 16```

Os pontos que o DLib retorna são zero based, portanto o primeiro ponto começa no 0.

In [ ]:
FACE_COMPLETA = list(range(0, 68))
SOMBRANCELHA_DIREITA = list(range(17, 22))
SOMBRANCELHA_ESQUERDA = list(range(22, 27))
OLHO_DIREITO = list(range(36, 42))
OLHO_ESQUERDO = list(range(42, 48))
NARIZ = list(range(27, 35))
LABIO = list(range(48, 61))
MANDIBULA = list(range(0, 17))

Função para criar polígonos a partir dos pontos de identificação. Estes polígonos podem oferecer ferramentas como cálculo de área para identificação melhor das estruturas faciais, como piscadas, movimentação da boca, dentre outros.

In [ ]:
def anotar_marcos_faciais_componente(imagem, marcos_faciais):

    if marcos_faciais is None:
        print("Não foi identificado nenhum marco facial.")
        return imagem
    
    for marco_facial in marcos_faciais:
        for k, d in enumerate(marco_facial):
            pontos = cv2.convexHull(marco_facial[FACE_COMPLETA])
            cv2.drawContours(imagem, [pontos], 0, (0, 255, 0), 2)

            pontos = cv2.convexHull(marco_facial[NARIZ])
            cv2.drawContours(imagem, [pontos], 0, (0, 255, 0), 2)

            pontos = cv2.convexHull(marco_facial[LABIO])
            cv2.drawContours(imagem, [pontos], 0, (0, 255, 0), 2)

            pontos = cv2.convexHull(marco_facial[SOMBRANCELHA_DIREITA])
            cv2.drawContours(imagem, [pontos], 0, (0, 255, 0), 2)

            pontos = cv2.convexHull(marco_facial[SOMBRANCELHA_ESQUERDA])
            cv2.drawContours(imagem, [pontos], 0, (0, 255, 0), 2)

            pontos = cv2.convexHull(marco_facial[OLHO_ESQUERDO])
            cv2.drawContours(imagem, [pontos], 0, (0, 255, 0), 2)

            pontos = cv2.convexHull(marco_facial[OLHO_DIREITO])
            cv2.drawContours(imagem, [pontos], 0, (0, 255, 0), 2)

    return imagem

In [ ]:
imagem = cv2.imread("imagens/cervero.jpeg")

marcos_faciais = obter_marcos(imagem)
imagem_marcos = anotar_marcos_faciais_componente(imagem, marcos_faciais)

imagem_marcos = cv2.cvtColor(imagem_marcos, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(30,20))
plt.axis('off')
plt.imshow(imagem_marcos)
plt.title("Marcos faciais com 68 pontos")

In [ ]:
imagem = cv2.imread("imagens/girl.jpg")

marcos_faciais = obter_marcos(imagem)
imagem_marcos = anotar_marcos_faciais_componente(imagem, marcos_faciais)

imagem_marcos = cv2.cvtColor(imagem_marcos, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(30,20))
plt.axis('off')
plt.imshow(imagem_marcos)
plt.title("Marcos faciais com 68 pontos")

Para lidar com situações onde o rosto capturado pode se aproximar ou se afastar da câmera utilizamos um cálculo de aspecto de razão. Esse cálculo divide duas medidas e sua razão é utilizada como medida para análises.

Uma vez que temos os pontos em volta de cada componente do rosto humano é possível realizar essas medidas de aspecto de razão para analisar a abertura dos olhos, por exemplo.

Nesse sentido, calculamos a medida de aspecto razão dos olhos ou EAR. Essa medida foi discutida neste [paper de Soukupová and Čech](http://vision.fe.uni-lj.si/cvww2016/proceedings/papers/05.pdf).

In [ ]:
def ear(pontos_olho):
    
    a = dist.euclidean(pontos_olho[1], pontos_olho[5])
    b = dist.euclidean(pontos_olho[2], pontos_olho[4])
    c = dist.euclidean(pontos_olho[0], pontos_olho[3])

    medida_ear = (a + b) / (2.0 * c)

    return medida_ear

Vamos inspecionar a abertura dos olhos da imagem analisada. Note que como existe apenas uma única pessoa (ou rosto) na imagem,  utilizaremos o índice 0 somente. Se houvessem mais pessoas, poderíamos analisar cada uma delas iterando sobre o índice.

In [ ]:
ear_olho_direito = ear(marcos_faciais[0][OLHO_DIREITO])
ear_olho_esquerdo = ear(marcos_faciais[0][OLHO_ESQUERDO])

print("EAR do olho esquerdo " + str(ear_olho_esquerdo))
print("EAR do olho direito " + str(ear_olho_direito))

In [ ]:
imagem = cv2.imread("imagens/mulher-olhos-fechados.jpg")

marcos_faciais = obter_marcos(imagem)
imagem_marcos = anotar_marcos_faciais_componente(imagem, marcos_faciais)

imagem_marcos = cv2.cvtColor(imagem_marcos, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(30,20))
plt.axis('off')
plt.imshow(imagem_marcos)
plt.title("Marcos faciais com 68 pontos")

In [ ]:
ear_olho_direito = ear(marcos_faciais[0][OLHO_DIREITO])
ear_olho_esquerdo = ear(marcos_faciais[0][OLHO_ESQUERDO])

print("EAR do olho esquerdo " + str(ear_olho_esquerdo))
print("EAR do olho direito " + str(ear_olho_direito))

Aplicar a análise de face e marcos faciais utilizando a câmera.

In [ ]:
from IPython.display import Image
try:
  filename = take_photo("imagens/foto.jpg")
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
imagem = cv2.imread("imagens/foto.jpg")

marcos_faciais = obter_marcos(imagem)
imagem_marcos = anotar_marcos_faciais_componente(imagem, marcos_faciais)

imagem_marcos = cv2.cvtColor(imagem_marcos, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(30,20))
plt.axis('off')
plt.imshow(imagem_marcos)
plt.title("Marcos faciais com 68 pontos")

In [ ]:
ear_olho_direito = ear(marcos_faciais[0][OLHO_DIREITO])
ear_olho_esquerdo = ear(marcos_faciais[0][OLHO_ESQUERDO])

print("EAR do olho esquerdo " + str(ear_olho_esquerdo))
print("EAR do olho direito " + str(ear_olho_direito))

### 3.2 Troca de rostos

Troca de rostos. Inicalmente utilizamos os marcadores da face para recortar adequadamente os rotos e posteriormente implantamos em outra imagem, aplicando suavizações em suas bordas.

_Retirado integralmente de [Matthew Earl](http://matthewearl.github.io/2015/07/28/switching-eds-with-python/)._

In [ ]:
import cv2
import dlib
import numpy
from time import sleep
import sys

SCALE_FACTOR = 1 
FEATHER_AMOUNT = 11

FACE_POINTS = list(range(17, 68))
MOUTH_POINTS = list(range(48, 61))
RIGHT_BROW_POINTS = list(range(17, 22))
LEFT_BROW_POINTS = list(range(22, 27))
RIGHT_EYE_POINTS = list(range(36, 42))
LEFT_EYE_POINTS = list(range(42, 48))
NOSE_POINTS = list(range(27, 35))
JAW_POINTS = list(range(0, 17))

# Points used to line up the images.
ALIGN_POINTS = (LEFT_BROW_POINTS + RIGHT_EYE_POINTS + LEFT_EYE_POINTS + RIGHT_BROW_POINTS + NOSE_POINTS + MOUTH_POINTS)

# Points from the second image to overlay on the first. The convex hull of each
# element will be overlaid.
OVERLAY_POINTS = [
    LEFT_EYE_POINTS + RIGHT_EYE_POINTS + LEFT_BROW_POINTS + RIGHT_BROW_POINTS,
    NOSE_POINTS + MOUTH_POINTS,
]

# Amount of blur to use during colour correction, as a fraction of the
# pupillary distance.
COLOUR_CORRECT_BLUR_FRAC = 0.6

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(classificador_68_path)

def get_landmarks(im):
    # Returns facial landmarks as (x,y) coordinates
    rects = detector(im, 1)
    
    if len(rects) > 1:
        return im
    if len(rects) == 0:
        return im

    return numpy.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])


def annotate_landmarks(im, landmarks):
    #Overlays the landmark points on the image itself
    
    im = im.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(im, str(idx), pos,
                    fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                    fontScale=0.4,
                    color=(0, 0, 255))
        cv2.circle(im, pos, 3, color=(0, 255, 255))
    return im

def draw_convex_hull(im, points, color):
    points = cv2.convexHull(points)
    cv2.fillConvexPoly(im, points, color=color)

def get_face_mask(im, landmarks):
    im = numpy.zeros(im.shape[:2], dtype=numpy.float64)

    for group in OVERLAY_POINTS:
        draw_convex_hull(im,
                         landmarks[group],
                         color=1)

    im = numpy.array([im, im, im]).transpose((1, 2, 0))

    im = (cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0) > 0) * 1.0
    im = cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0)

    return im
    
def transformation_from_points(points1, points2):
    # Solve the procrustes problem by subtracting centroids, scaling by the
    # standard deviation, and then using the SVD to calculate the rotation. See
    # the following for more details:
    #   https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem

    points1 = points1.astype(numpy.float64)
    points2 = points2.astype(numpy.float64)

    c1 = numpy.mean(points1, axis=0)
    c2 = numpy.mean(points2, axis=0)
    points1 -= c1
    points2 -= c2

    s1 = numpy.std(points1)
    s2 = numpy.std(points2)
    points1 /= s1
    points2 /= s2

    U, S, Vt = numpy.linalg.svd(points1.T * points2)

    # The R we seek is in fact the transpose of the one given by U * Vt. This
    # is because the above formulation assumes the matrix goes on the right
    # (with row vectors) where as our solution requires the matrix to be on the
    # left (with column vectors).
    R = (U * Vt).T

    return numpy.vstack([numpy.hstack(((s2 / s1) * R,
                                       c2.T - (s2 / s1) * R * c1.T)),
                         numpy.matrix([0., 0., 1.])])

def read_im_and_landmarks(image):
    im = image
    im = cv2.resize(im,None,fx=1, fy=1, interpolation = cv2.INTER_LINEAR)
    im = cv2.resize(im, (im.shape[1] * SCALE_FACTOR,
                         im.shape[0] * SCALE_FACTOR))
    s = get_landmarks(im)

    return im, s

def warp_im(im, M, dshape):
    output_im = numpy.zeros(dshape, dtype=im.dtype)
    cv2.warpAffine(im,
                   M[:2],
                   (dshape[1], dshape[0]),
                   dst=output_im,
                   borderMode=cv2.BORDER_TRANSPARENT,
                   flags=cv2.WARP_INVERSE_MAP)
    return output_im

def correct_colours(im1, im2, landmarks1):
    blur_amount = COLOUR_CORRECT_BLUR_FRAC * numpy.linalg.norm(
                              numpy.mean(landmarks1[LEFT_EYE_POINTS], axis=0) -
                              numpy.mean(landmarks1[RIGHT_EYE_POINTS], axis=0))
    blur_amount = int(blur_amount)
    if blur_amount % 2 == 0:
        blur_amount += 1
    im1_blur = cv2.GaussianBlur(im1, (blur_amount, blur_amount), 0)
    im2_blur = cv2.GaussianBlur(im2, (blur_amount, blur_amount), 0)

    # Avoid divide-by-zero errors.
    im2_blur += (128 * (im2_blur <= 1.0)).astype(im2_blur.dtype)

    return (im2.astype(numpy.float64) * im1_blur.astype(numpy.float64) /
                                                im2_blur.astype(numpy.float64))

def swappy(image1, image2):
       
    im1, landmarks1 = read_im_and_landmarks(image1)
    im2, landmarks2 = read_im_and_landmarks(image2)

    M = transformation_from_points(landmarks1[ALIGN_POINTS],
                                   landmarks2[ALIGN_POINTS])
    
    mask = get_face_mask(im2, landmarks2)
    warped_mask = warp_im(mask, M, im1.shape)
    combined_mask = numpy.max([get_face_mask(im1, landmarks1), warped_mask],
                              axis=0)

    warped_im2 = warp_im(im2, M, im1.shape)
    warped_corrected_im2 = correct_colours(im1, warped_im2, landmarks1)

    output_im = im1 * (1.0 - combined_mask) + warped_corrected_im2 * combined_mask
    
    cv2.imwrite('imagens/aux_output.jpg', output_im)
    image = cv2.imread('imagens/aux_output.jpg')
    
    return image

Utilizando a câmera para obter uma imagem.

In [ ]:
from IPython.display import Image
try:
  filename = take_photo("imagens/foto.jpg")
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

Ou utilizar imagens já salvas.

In [ ]:
#image1 = cv2.imread('imagens/foto.jpg')

image1 = cv2.imread('imagens/faustao-swap.jpg')
image2 = cv2.imread('imagens/silvio-santos-swap.jpg')

swapped_1 = swappy(image1, image2)
swapped_2 = swappy(image2, image1)

swapped_1 = cv2.cvtColor(swapped_1, cv2.COLOR_BGR2RGB)
swapped_2 = cv2.cvtColor(swapped_2, cv2.COLOR_BGR2RGB)

swapped_1 = cv2.resize(swapped_1, (850, 600))
swapped_2 = cv2.resize(swapped_2, (850, 600))

plt.figure(figsize=(20,20))

plt.subplot(1,2,1)
plt.imshow(swapped_1)
plt.title("Fausto Santos")
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(swapped_2)
plt.title("Silvo Silva")
plt.axis('off')